# For kaggle downloading

In [1]:
!pip install gdown > /dev/null

In [2]:
# !gdown --id <File ID>
# https://drive.google.com/file/d/1Q2cOZEt8UJbwjJkztri6T74gY61LUT6p/view?usp=sharing
!gdown 1Q2cOZEt8UJbwjJkztri6T74gY61LUT6p

Downloading...
From (original): https://drive.google.com/uc?id=1Q2cOZEt8UJbwjJkztri6T74gY61LUT6p
From (redirected): https://drive.google.com/uc?id=1Q2cOZEt8UJbwjJkztri6T74gY61LUT6p&confirm=t&uuid=b779ff64-7ba4-4e38-a729-ff3d73b795d0
To: /kaggle/working/school_notebooks_RU.zip
100%|███████████████████████████████████████| 3.53G/3.53G [00:27<00:00, 126MB/s]


In [1]:
!git lfs install
!git clone https://huggingface.co/datasets/ai-forever/school_notebooks_EN /kaggle/working/school_notebooks_EN

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into '/kaggle/working/school_notebooks_EN'...
remote: Enumerating objects: 34, done.
remote: Total 34 (delta 0), reused 0 (delta 0), pack-reused 34
Unpacking objects: 100% (34/34), 3.98 MiB | 5.93 MiB/s, done.


# For google colab 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Track changes

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from getpass import getpass
token = getpass('GitHub PAT')
os.environ['PAT'] = token
!git clone https://$PAT@github.com/RuslanZaripov/rhtr.git

GitHub PAT ········································


Cloning into 'rhtr'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 257 (delta 169), reused 187 (delta 99), pack-reused 0
Receiving objects: 100% (257/257), 11.05 MiB | 25.42 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [48]:
%cd /kaggle/working/rhtr
!git restore .
!git pull
%cd /kaggle/working

/kaggle/working/rhtr
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), 402 bytes | 402.00 KiB/s, done.
From https://github.com/RuslanZaripov/rhtr
   52fa6d6..74ff513  main       -> origin/main
Updating 52fa6d6..74ff513
Fast-forward
 src/segmentation/dataset.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
/kaggle/working


# Unzip imported dataset into separate folder

In [4]:
!unzip -o -q '/kaggle/working/school_notebooks_EN/images.zip' -d '/kaggle/working/school_notebooks_EN' > /dev/null

In [5]:
!rm -rf '/kaggle/working/school_notebooks_EN/images.zip'

In [ ]:
# !cp -r /kaggle/input/processed-school-notebooks/* /kaggle/working

from distutils.dir_util import copy_tree
copy_tree("/kaggle/input/processed-school-notebooks-ru", "/kaggle/working")

In [8]:
!unzip -o -q '/kaggle/working/school_notebooks_RU.zip' -d '/kaggle/working/school_notebooks_RU' > /dev/null

In [9]:
!rm -rf '/kaggle/working/school_notebooks_RU.zip'

# If you need to arrange folders correctly use the code below

In [10]:
!mkdir /kaggle/working/school_notebooks_RU # create folder where data will be stored

mkdir: cannot create directory '/kaggle/working/school_notebooks_RU': File exists


In [11]:
!cp -r /kaggle/working/school_notebooks_RU/content/magazine/* /kaggle/working/school_notebooks_RU

In [12]:
!rm -rf /kaggle/working/school_notebooks_RU/content/

# Upload annotations to /content/magazine and split them if needed

In [ ]:
!pip install funcy > /dev/null
!pip install scikit-multilearn > /dev/null

In [ ]:
!python /content/rhtr/src/segmentation/scripts/cocosplit.py --having-annotations --multi-class -s 0.8 /content/magazine/annotations.json /content/magazine/annotations_train.json /content/magazine/annotations_test.json

In [ ]:
!python /content/rhtr/src/segmentation/scripts/cocosplit.py --having-annotations --multi-class -s 0.8 /content/magazine/annotations_train.json /content/magazine/annotations_train.json /content/magazine/annotations_val.json

# Specify python path

In [7]:
import os
os.environ['PYTHONPATH'] += ":/kaggle/working/rhtr"

In [8]:
!echo $PYTHONPATH

/kaggle/lib/kagglegym:/kaggle/lib:/kaggle/working/rhtr


In [9]:
!pip install pyclipper > /dev/null

# Edit config file

In [49]:
write = '''
{
    "save_dir": "/kaggle/working/experiments/segm",
    "tensorboard_log_dir": "/kaggle/working/tensorboard/log",
    "num_epochs": 1500,
    "pretrain_path": "",
    "image": {
        "width": 896,
        "height": 896
    },
    "classes": {
        "class_name_1": {
            "annotation_classes": ["pupil_text", "pupil_comment", "teacher_comment"],
            "polygon2mask": {
                "ShrinkMaskMaker": {
                    "shrink_ratio": 0.8
                }
            },
            "postprocess": {
                "threshold": 0.3,
                "min_area": 10
            }
        },
        "class_name_2": {
            "annotation_classes": ["pupil_text", "pupil_comment", "teacher_comment"],
            "polygon2mask": {
                "BorderMaskMaker": {
                    "shrink_ratio": 0.5
                }
            },
            "postprocess": {
                "threshold": 0.8,
                "min_area": 10
            }
        },
        "class_name_3": {
            "annotation_classes": ["text_line"],
            "polygon2mask": {
                "PolylineToMask": {
                    "thickness": 10
                }
            },
            "postprocess": {
                "threshold": 0.8,
                "min_area": 1
            }
        }
    },
    "train": {
        "datasets": [
            {
                "json_path": "/kaggle/working/school_notebooks_RU/annotations_train.json",
                "image_root": "/kaggle/working/school_notebooks_RU/images/",
                "processed_data_path": "/kaggle/working/school_notebooks_RU/annotations_train.csv",
                "prob": 0.5
            },
            {
                "json_path": "/kaggle/working/school_notebooks_EN/annotations_train.json",
                "image_root": "/kaggle/working/school_notebooks_EN/images/",
                "processed_data_path": "/kaggle/working/school_notebooks_EN/annotations_train.csv",
                "prob": 0.5
            }
        ],
        "epoch_size": 125,
        "batch_size": 4
    },
    "val": {
        "datasets": [
            {
                "json_path": "/kaggle/working/school_notebooks_RU/annotations_val.json",
                "image_root": "/kaggle/working/school_notebooks_RU/images/",
                "processed_data_path": "/kaggle/working/school_notebooks_RU/annotations_val.csv",
                "prob": 0.5
            },
            {
                "json_path": "/kaggle/working/school_notebooks_EN/annotations_val.json",
                "image_root": "/kaggle/working/school_notebooks_EN/images/",
                "processed_data_path": "/kaggle/working/school_notebooks_EN/annotations_val.csv",
                "prob": 0.5
            }
        ],
        "epoch_size": null,
        "batch_size": 10
    },
    "test": {
        "datasets": [
            {
                "json_path": "/kaggle/working/school_notebooks_RU/annotations_test.json",
                "image_root": "/kaggle/working/school_notebooks_RU/images/",
                "processed_data_path": "/kaggle/working/school_notebooks_RU/annotations_test.csv",
                "prob": 0.5
            },
            {
                "json_path": "/kaggle/working/school_notebooks_EN/annotations_test.json",
                "image_root": "/kaggle/working/school_notebooks_EN/images/",
                "processed_data_path": "/kaggle/working/school_notebooks_EN/annotations_test.csv",
                "prob": 0.5
            }
        ],
        "epoch_size": null,
        "batch_size": 10
    }
}
'''
with open("/kaggle/working/rhtr/src/segmentation/configs/segm_kaggle.json", "w") as file:
    file.write(write)

In [50]:
!cat /kaggle/working/rhtr/src/segmentation/configs/segm_kaggle.json


{
    "save_dir": "/kaggle/working/experiments/segm",
    "tensorboard_log_dir": "/kaggle/working/tensorboard/log",
    "num_epochs": 1500,
    "pretrain_path": "",
    "image": {
        "width": 896,
        "height": 896
    },
    "classes": {
        "class_name_1": {
            "annotation_classes": ["pupil_text", "pupil_comment", "teacher_comment"],
            "polygon2mask": {
                "ShrinkMaskMaker": {
                    "shrink_ratio": 0.8
                }
            },
            "postprocess": {
                "threshold": 0.3,
                "min_area": 10
            }
        },
        "class_name_2": {
            "annotation_classes": ["pupil_text", "pupil_comment", "teacher_comment"],
            "polygon2mask": {
                "BorderMaskMaker": {
                    "shrink_ratio": 0.5
                }
            },
            "postprocess": {
                "threshold": 0.8,
                "min_area": 10
            }
        },
        "

# Preprocess data

# REMEMBER TO CHANGE CONFIGURATION AFTER PREPROCESSING

In [22]:
!python /kaggle/working/rhtr/src/segmentation/scripts/prepare_dataset.py --config_path /kaggle/working/rhtr/src/segmentation/configs/segm_kaggle.json

cwd='/kaggle/working' config_path='/kaggle/working/rhtr/src/segmentation/configs/segm_kaggle.json'
  4%|█▉                                          | 3/70 [00:01<00:36,  1.83it/s]/kaggle/working/rhtr/src/segmentation/dataset.py:296: RuntimeWarning: invalid value encountered in divide
  u = u1 / (np.square(x1 - x2) + np.square(y1 - y2))
100%|███████████████████████████████████████████| 10/10 [00:07<00:00,  1.43it/s]


# Save preprocessed data to zip folder and upload to google drive

In [ ]:
!zip -r /content/school_notebooks_RU.zip /content/magazine > /dev/null

In [ ]:
import shutil
shutil.copy("/content/school_notebooks_RU.zip","/content/drive/MyDrive/data")

# Start train process

In [13]:
!pip install tensorboardX > /dev/null

In [98]:
!rm -rf '/kaggle/working/experiments/segm'
!rm -rf '/kaggle/working/tensorboard/log'

In [93]:
%matplotlib inline
import matplotlib.pyplot as plt

In [99]:
write = '''
import torch
import os
import math
import logging
import time
import numpy as np

from tqdm import tqdm

from src.segmentation.metrics import get_iou, get_f1_score, AverageMeter, IOUMetric, dice_loss
from src.segmentation.predictor import predict


def configure_logging(log_path=None):
    logger = logging.getLogger(__name__)

    if len(logger.handlers) > 0:
        return logger

    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        fmt='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%d-%b-%y %H:%M:%S'
    )
    # Setup console logging
    sh = logging.StreamHandler()
    sh.setLevel(logging.DEBUG)
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    # Setup file logging as well
    if log_path is not None:
        fh = logging.FileHandler(log_path)
        fh.setLevel(logging.DEBUG)
        fh.setFormatter(formatter)
        logger.addHandler(fh)
    return logger


def val_loop(data_loader, model, criterion, device, epoch, class_names, logger, writer=None):
    loss_avg = AverageMeter('Loss', ':.4e')
    iou = AverageMeter('IOU', ':6.2f')
    cls2iou = {class_name: IOUMetric(class_idx)
               for class_idx, class_name in enumerate(class_names)}
    f1_score_avg = AverageMeter('F1 score', ':6.2f')
    strat_time = time.time()

    tqdm_data_loader = tqdm(data_loader, total=len(data_loader), leave=False)
    for images, targets in tqdm_data_loader:
        preds, targets = predict(images, model, device, targets)
        batch_size = len(images)

        thresh_binary = preds[:, -1:, :, :]
        preds = preds[:, :-1, :, :]

        loss = criterion(preds, targets) + dice_loss(thresh_binary, targets[:, 0, :, :])

        loss_avg.update(loss.item(), batch_size)
        iou.update(get_iou(preds, targets), batch_size)
        f1_score_avg.update(get_f1_score(preds, targets), batch_size)
        for cls_name in class_names:
            cls2iou[cls_name].update(preds, targets)

    loop_time = sec2min(time.time() - strat_time)
    cls2iou_log = ', '.join([f'{class_name}: {iou}'
                             for class_name, iou in cls2iou.items()])
    logger.info(f'Validation: {loss_avg}, {iou}, {cls2iou_log}, {f1_score_avg}, loop_time: {loop_time}')

    if writer is not None:
        writer.add_scalar('Loss/val', loss_avg.avg, epoch)
        writer.add_scalar('IOU/val', iou.avg, epoch)
        writer.add_scalar('F1_score/val', f1_score_avg.avg, epoch)
        writer.add_scalars('IOU/val_cls', {cls_name: iou_fun.avg() for cls_name, iou_fun in cls2iou.items()}, epoch)

    return loss_avg.avg


def sec2min(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


class FilesLimitControl:
    """Delete files from the disk if there are more files than the set limit.
    Args:
        max_weights_to_save (int, optional): The number of files that will be
            stored on the disk at the same time. Default is 3.
    """

    def __init__(self, logger=None, max_weights_to_save=2):
        self.saved_weights_paths = []
        self.max_weights_to_save = max_weights_to_save
        self.logger = logger if logger is not None else configure_logging()

    def __call__(self, save_path):
        self.saved_weights_paths.append(save_path)
        if len(self.saved_weights_paths) > self.max_weights_to_save:
            old_weights_path = self.saved_weights_paths.pop(0)
            if os.path.exists(old_weights_path):
                os.remove(old_weights_path)
                self.logger.info(f"Weigths removed '{old_weights_path}'")


def load_pretrain_model(weights_path, model, logger=None):
    """Load the entire pretrain model or as many layers as possible.
    """
    old_dict = torch.load(weights_path)
    new_dict = model.state_dict()
    if logger is None:
        logger = configure_logging()
    for key, weights in new_dict.items():
        if key in old_dict:
            if new_dict[key].shape == old_dict[key].shape:
                new_dict[key] = old_dict[key]
            else:
                logger.info('Weights {} were not loaded'.format(key))
        else:
            logger.info('Weights {} were not loaded'.format(key))
    return new_dict


"""
Source: https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch

Early stops the training if validation loss doesn't improve after a given patience.
Source: https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
Usage: https://github.com/Bjarten/early-stopping-pytorch/blob/master/MNIST_Early_Stopping_example.ipynb
 
Another source: https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_03_4_early_stop.ipynb
where best model restoring happens as a feature
"""


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0,
                 logger=None,
                 load_best_weights=True,
                 control_files_limit=True):
        """
        Args:
            patience (int):
                How long to wait after last time validation loss improved.
                Default: 7
            min_delta (float):
                Minimum change in the monitored quantity to qualify as an improvement.
                Should be a positive number.
                Default: 0
            logger (logging.Logger):
                Logger object to use for logging. If None, no logging is done.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0

        self.load_best_weights = load_best_weights
        self.logger = logger if logger is not None else configure_logging()
        self.file_limit_control = FilesLimitControl(logger=self.logger) if control_files_limit else None

    def is_loss_decreased(self, val_loss):
        return self.best_loss - val_loss >= self.min_delta

    def __call__(self, val_loss, checkpoint_path, model):
        if self.is_loss_decreased(val_loss):
            self.log_info(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')
            self.best_loss = val_loss
            self.counter = 0

            self.save_checkpoint(model, checkpoint_path)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.log_info(f'EarlyStopping triggered after {self.counter} epochs.')
                if self.load_best_weights:
                    model.load_state_dict(torch.load(checkpoint_path))
                return True
        return False

    def log_info(self, message):
        if self.logger is not None:
            self.logger.info(message)

    def save_checkpoint(self, model, checkpoint_path):
        """Saves model when validation loss decrease."""
        torch.save(model.state_dict(), checkpoint_path)
        self.log_info(f'Model saved to {checkpoint_path}')
        if self.file_limit_control is not None:
            self.file_limit_control(checkpoint_path)

'''
with open("/kaggle/working/rhtr/src/segmentation/utils.py", "w") as file:
    file.write(write)

In [55]:
import os
os.chdir("/kaggle/working/school_notebooks_RU/targets")
os.path.exists('/kaggle/working/school_notebooks_RU/targets\2764.npy')
# os.listdir() # to see files in this directory

False

In [ ]:
!cat /kaggle/working/rhtr/src/segmentation/dataset.py

In [100]:
!python /kaggle/working/rhtr/src/segmentation/scripts/train.py --config_path /kaggle/working/rhtr/src/segmentation/configs/segm_kaggle.json

cwd='/kaggle/working/school_notebooks_RU/targets' config_path='/kaggle/working/rhtr/src/segmentation/configs/segm_kaggle.json'
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid 

# Load tensorboard

In [102]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
!kill 4829

In [104]:
%tensorboard --logdir '/kaggle/working/tensorboard/log'

In [ ]:
!find '/content/tensorboard/log' | grep tfevents

In [ ]:
!tensorboard --inspect --logdir '/content/tensorboard/log'

# Interact with a git repo

In [ ]:
%cd /kaggle/working

In [ ]:
!git restore .
!git pull

In [ ]:
!git status

In [ ]:
!git diff > /content/diff_rhtr.txt

# Zip experiments/ and tensorboard/ folders

In [111]:
!zip -r /kaggle/working/experiments.zip /kaggle/working/experiments

  adding: kaggle/working/experiments/ (stored 0%)
  adding: kaggle/working/experiments/segm/ (stored 0%)
  adding: kaggle/working/experiments/segm/2024-03-01_11-47-30/ (stored 0%)
  adding: kaggle/working/experiments/segm/2024-03-01_11-47-30/LinkResNet-6-0.4946.ckpt (deflated 7%)
  adding: kaggle/working/experiments/segm/2024-03-01_11-47-30/LinkResNet-8-0.4719.ckpt (deflated 7%)
  adding: kaggle/working/experiments/segm/2024-03-01_11-47-30/output.log (deflated 82%)


In [112]:
!zip -r /kaggle/working/tensorboard.zip /kaggle/working/tensorboard

  adding: kaggle/working/tensorboard/ (stored 0%)
  adding: kaggle/working/tensorboard/log/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cls/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cls/class_name_2/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cls/class_name_2/events.out.tfevents.1709293668.ec1ded4a21ee (deflated 53%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cls/class_name_1/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cls/class_name_1/events.out.tfevents.1709293668.ec1ded4a21ee (deflated 54%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cls/class_name_3/ (stored 0%)
  adding: kaggle/working/tensorboard/log/2024-03-01_11-47-30/IOU/val_cl

In [113]:
!zip -r /kaggle/working/images.zip /kaggle/working/images

  adding: kaggle/working/images/ (stored 0%)
  adding: kaggle/working/images/0/ (stored 0%)
  adding: kaggle/working/images/0/pred_1.jpg (deflated 10%)
  adding: kaggle/working/images/0/pred_0.jpg (deflated 13%)
  adding: kaggle/working/images/0/thresh_binary.jpg (deflated 10%)
  adding: kaggle/working/images/0/pred_2.jpg (deflated 14%)


In [109]:
from IPython.display import FileLink
FileLink(r'experiments.zip')

/kaggle/working/school_notebooks_RU/targets/experiments.zip